# Análise de texto de fontes desestruturadas e Web

## Aula 11

Nesta aula vamos fazer uma aplicação prática de Machine Learning.

A biblioteca utilizada será a **scikit-learn**.

Para conhecer mais sobre ela, acesse https://scikit-learn.org/stable/tutorial/basic/tutorial.html

## Baixar os dados no Colab
Para baixar os dados no colab, utilize

In [ ]:
!wget https://atd-insper.s3.us-east-2.amazonaws.com/aula10/noticias.csv
!wget https://atd-insper.s3.us-east-2.amazonaws.com/aula10/noticias.xlsx

## Importando as bibliotecas necessárias

Agora, vamos importar as bibliotecas necessárias:

In [ ]:
# para trabalhar com diretórios / sistema operacional
import os

# para trabalhar com expressões regulares
import re

# utilizada para nos indicar o caminho do executável do Python
import sys

# para pandas DataFrame
import pandas as pd

# ML
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Vectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Pipeline
from sklearn.pipeline import Pipeline

Caso obtenha algum erro, utilize o **!pip install** para instalar a biblioteca ausente!

Você pode conferir de onde está executando o Python e qual a versão

In [ ]:
print('Executável:')
print(sys.executable)

print('\nVersão do Python:')
print(sys.version)

Vamos conferir em qual diretório iremos trabalhar (é o diretório do notebook)

In [ ]:
print('O seu notebook está na pasta:')
print(os.getcwd())

# Text Vectorization

Vamos aprender como transformar textos de forma a conseguir treinar modelos a partir deles utilizando o modelo **Bag of Words (BoW)**. No python, o BoW já está implementado como CountVectorizer na biblioteca **sklearn**.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Considere o dataframe de exemplo

In [ ]:
df = pd.DataFrame(
    {
        "Texto": [
            "quero ir para Praia",
            "não gosto de praia, só de sol",
            "quero sombra e água",
        ]
    }
)
df


Primeiro, criamos um CountVectorizer

In [ ]:
exvec = CountVectorizer(binary=True)

E mandamos ele se *ajustar* aos dados, ou seja, aprender o vocabulário (fit) e já devoltar os dados de treinamento transformados (transform).

In [ ]:
mat = exvec.fit_transform(df['Texto']).toarray()
mat

In [ ]:
exvec.get_feature_names_out()

In [ ]:
df_vec = pd.DataFrame(mat)
df_vec.columns = exvec.get_feature_names_out()
df_vec

### Utilizando com dados novos

Agora, podemos utilizar o CountVectorizer com dados novos (frases ainda não vistas).

In [ ]:
df

Primeiro exemplo utilizando uma frase que já pertencia a base

In [ ]:
txt = 'quero sombra e água'
exvec.transform([txt]).toarray()

Modificando um pouco a frase

In [ ]:
txt = 'em copacabana quero sombra e água'
exvec.transform([txt]).toarray()

In [ ]:
txt = '' # algum texto aqui!
exvec.transform([txt]).toarray()

In [ ]:
exvec.get_feature_names_out()

Por que ele não modificou?

R:

# Notícias
## Carregando os dados

Vamos abrir os dados que estão armazenados em planilhas. Você pode escolher a versão CSV ou XLSX

In [ ]:
df = pd.read_csv('noticias.csv')

Caso prefira utilizar a versão Excel, abra com

In [ ]:
df = pd.read_excel('noticias.xlsx')

Vamos ver o que temos armazenado no DataFrame?!

In [ ]:
df.head(10)

Quantas notícias temos ao todo?

In [ ]:
df.shape

Quantas seções de notícias temos? E quantas notícias por seção?

In [ ]:
df['Secao'].value_counts()

# Separando em Treino e Teste

Para simular uma situação real de uso do classificador e termos uma ideia de como o classificador se sairá em um uso futuro, iremos dividir nossa base de dados em duas, utilizando uma base para treinar o modelo e outra base para avaliar o desempenho do modelo treinado.

In [ ]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(df[['Titulo', 'Descrição']],
                                                            df['Secao'],
                                                            test_size=0.25,
                                                            random_state=151)

Conferindo os dados de treinamento (X e y)

In [ ]:
X_train_raw

In [ ]:
y_train

## Extração de Features

Não vamos utilizar direto o título para treino de um modelo. A partir do título, iremos criar variáveis com o uso da técnica de **Bag of Words**.

Primeiro, criamos um CountVectorizer

In [ ]:
cvec = CountVectorizer()

E *ajustar* ele aos dados, ou seja, fazer aprender o vocabulário (fit) e já devoltar os dados de treinamento transformados (transform)

In [ ]:
mat = cvec.fit_transform(X_train_raw['Titulo']).toarray()

In [ ]:
X_train = pd.DataFrame(mat)
X_train.columns = cvec.get_feature_names_out()
X_train.sample(20)

Agora com os dados de teste

In [ ]:
X_test_raw

In [ ]:
mat_test = cvec.transform(X_test_raw['Titulo']).toarray()

In [ ]:
X_test = pd.DataFrame(mat_test)
X_test.columns = cvec.get_feature_names_out()
X_test.sample(2)

In [ ]:
X_test.head(2)

## Criando o classificador
Primeiro, vamos utilizar um **DecisionTreeClassifier**

In [ ]:
dt = DecisionTreeClassifier()

Para treiná-lo, basta fazer:

In [ ]:
dt.fit(X_train, y_train)

## Base de testes
Agora, vamos ver a performance na base de testes

In [ ]:
X_test.shape

In [ ]:
y_pred = dt.predict(X_test)
accuracy_score(y_true=y_test, y_pred=y_pred)

## Utilizando Pipeline
Agora, vamos ver a performance na base de testes

In [ ]:
pipe = Pipeline(
    [
        ("vect", CountVectorizer(binary=True)),
        ("rf", RandomForestClassifier(n_estimators=100, n_jobs=-1)),
    ]
)

pipe.fit(X_train_raw["Descrição"], y_train)

Performance nos dados de treinamento:

In [ ]:
y_pred = pipe.predict(X_train_raw['Descrição'])
accuracy_score(y_true=y_train, y_pred=y_pred)

Performance nos dados de teste:

In [ ]:
y_pred = pipe.predict(X_test_raw['Descrição'])
accuracy_score(y_true=y_test, y_pred=y_pred)

# Exercícios

**Exercício 1)** Veja na documentação do `CountVectorizer` o parâmetro `stop_words`. Passe uma lista de palavras para este parâmetro. Explique o que acontece.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Dica: procure no Google por `nltk stopwords portuguese`

R:

**Exercício 2)** Veja na documentação do CountVectorizer o parâmetro `vocabulary`. Passe uma lista de palavras para este parâmetro. Explique o que acontece.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

**Exercício 3)** Veja na documentação do CountVectorizer o parâmetro `binary`. O que acontece se treinarmos um CountVectorizer com `binary=False`? Que impactos, positivos ou negativos, isto pode trazer ao nosso modelo?

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

**Exercício 4)** Considerando a versão atualizada do seu `CountVectorizer`, verifique a performance de outros modelos. Algumas sugestões:

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

**Exercício 5)** No `CountVectorizer`, consideramos apenas as palavras isoladamente. Entretanto, a ocorrência de certas palavras após outra determinada palavra também é algo informativo. Confira o parâmetro `ngram_range` na documentação do `CountVectorizer`. Teste algumas variações para o `ngram_range` e analise os resultados.

**Exercício 6)** Existem outras formas de implementar um Vectorizer. Uma opção bastante utilizada é o `tf-idf`.

Confira tutoriais em:

http://www.tfidf.com

https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76

E a documentação oficial do sk-lean:

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

Então, utilize o `TfidfVectorizer` para vetorização dos testes e avalie sua performance.

**Exercício 7)** Os modelos baseados em árvore também podem ser utilizados para seleção de features. Treine um `DecisionTreeClassifier` utilizando uma profundidade baixa (Ex: 4). Desenhe a árvore (aula 09) e confira quais features foram utilizadas para separação das amostras.